# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st

In [2]:
df0 = pd.read_csv('/Users/vasily/Learning/Data Analytics/Projects/Oil/geo_data_0.csv')
df1 = pd.read_csv('/Users/vasily/Learning/Data Analytics/Projects/Oil/geo_data_1.csv')
df2 = pd.read_csv('/Users/vasily/Learning/Data Analytics/Projects/Oil/geo_data_2.csv')

print(df0.sample(n=10, random_state=123))
print(df0.info())
print(df0.describe())

print(df1.sample(n=10, random_state=123))
print(df1.info())
print(df1.describe())

print(df2.sample(n=10, random_state=123))
print(df2.info())
print(df2.describe())

          id        f0        f1        f2     product
42083  qdC6b  0.331011 -0.288668  6.134432  145.825972
71825  DmmHg -0.027196  0.189525  0.160777  134.020212
99535  Yjids -0.982324  0.286965 -2.163846   88.905826
47879  xZBy3  0.686155  0.797873  2.689418   19.508226
36734  k7gBv  0.603928  0.605621  5.852418  108.713468
48636  rKujK -0.055130  0.810778  2.929756  105.325355
59566  X1D5k  0.316299 -0.389996  3.323526  100.696868
44826  Tczap  1.393358 -0.462688  3.827630  100.733884
51584  plPop  0.138965 -0.182593  4.965173  133.570355
4368   6IyO5  1.816823  0.027294  6.109931  112.674393
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float

In [3]:
#Уберем колонку id как не релевантную для обучения модели
df0 = df0.drop(['id'], axis=1)
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)
print(df0.info())
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float6

**Вывод**

Выполнены действия по подготовке данных:
- Загрузка из файлов в три датафрейма, по одному на каждый регион
- Вывод информации о колонках. Все признаки в данных числовые, кроме колонки id, которую мы убрали как нерелевантную для обучения
- Просмотр статистики по значениям признаков. Все три признака имеют одинаковый масштаб, так что нет необходимости в приведении к единому масштабу.

## Обучение и проверка модели

In [4]:
def model(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state=123)
    model=LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    return target_valid, predicted_valid, rmse
    

In [5]:
#Обучение и проверка модели для региона 0

features = df0.drop(['product'], axis=1)
target = df0['product']
target_valid_0, predicted_valid_0, rmse = model(features, target)
print('Средний запас предсказанного сырья для скважин региона 0, тыс.баррелей:', predicted_valid_0.mean())
print('RMSE модели для региона 0:', rmse)

#Обучение и проверка модели для региона 1
features = df1.drop(['product'], axis=1)
target = df1['product']
target_valid_1, predicted_valid_1, rmse = model(features, target)
print('Средний запас предсказанного сырья для скважин региона 1, тыс.баррелей:', predicted_valid_1.mean())
print('RMSE модели для региона 1:', rmse)

#Обучение и проверка модели для региона 2
features = df2.drop(['product'], axis=1)
target = df2['product']
target_valid_2, predicted_valid_2, rmse = model(features, target)
print('Средний запас предсказанного сырья для скважин региона 2, тыс.баррелей:', predicted_valid_2.mean())
print('RMSE модели для региона 2:', rmse)

Средний запас предсказанного сырья для скважин региона 0, тыс.баррелей: 92.54936189116309
RMSE модели для региона 0: 37.64786282376176
Средний запас предсказанного сырья для скважин региона 1, тыс.баррелей: 69.28001860653978
RMSE модели для региона 1: 0.8954139804944308
Средний запас предсказанного сырья для скважин региона 2, тыс.баррелей: 95.09859933591373
RMSE модели для региона 2: 40.12803006598514


**Вывод**

Среднеквадратичная шибка моделей по регионам 0 и 2 получилась высокая, почти я 1/2 от среднего предсказанного запаса сырья. Точность же модели в регионе 1 получилась высокая, при том что средний запас там самый низкий.

## Подготовка к расчёту прибыли

In [6]:
EXPLORED_WELLS_N = 500 #Кол-во потенциальных точек скважин, исследоваемых в каждом регионе
BEST_WELLS_N = 200 #Кол-во лучших точек скважин, выбираемых для разработки
COST = 10000000 #Бюджет на разработку скважин в регионе, тыс.руб
EARNINGS = 450 #стоимость с единицы продукта, тыс.руб

min_well_product = COST/(BEST_WELLS_N * EARNINGS) #достаточный объём сырья на скважине для безубыточной разработки новой скважины
print('Достаточный объём сырья на скважине для безубыточной разработки новой скважины, тыс.баррелей:',min_well_product)

Достаточный объём сырья на скважине для безубыточной разработки новой скважины, тыс.баррелей: 111.11111111111111


## Расчёт прибыли и рисков 

In [7]:
 #Функция расчета прибыли
def profit(predicted, target, wells_count):
    predicted = pd.Series(predicted).reset_index(drop=True)
    target = pd.Series(target).reset_index(drop=True)
    
    predicted_sorted = predicted.sort_values(ascending=False)
    selected = target[predicted_sorted.index][:wells_count]
    profit = selected.sum()*EARNINGS - COST
    
    return profit

In [8]:
#Функция Bootstrap и анализа

def bootstrap(predicted, target, n=1000):
    target = target.reset_index(drop=True)
    state = np.random.RandomState(123)
    values = []
    for i in range (n):
        target_subsample = target.sample(n=EXPLORED_WELLS_N, replace=True, random_state=state)
        predicted_subsample = predicted[target_subsample.index]
        values.append(profit(predicted_subsample, target_subsample, BEST_WELLS_N))
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = [values.quantile(0.025), values.quantile(0.975)]
    risk = (values<0).mean()
    
    print('Средняя прибыль:', mean)
    print('Доверительный интервал прибыли:', confidence_interval)
    print('Риск убытков:', risk)

In [9]:
#Анализ для региона 0
bootstrap(predicted_valid_0, target_valid_0)

#Анализ для региона 1
bootstrap(predicted_valid_1, target_valid_1)

#Анализ для региона 2
bootstrap(predicted_valid_2, target_valid_2)

Средняя прибыль: 477416.8242664122
Доверительный интервал прибыли: [-57993.91179927535, 974822.0147728735]
Риск убытков: 0.041
Средняя прибыль: 479190.16130033205
Доверительный интервал прибыли: [58726.87758795573, 874424.8194881873]
Риск убытков: 0.009
Средняя прибыль: 343454.376580876
Доверительный интервал прибыли: [-231376.25522789144, 860840.6698830121]
Риск убытков: 0.099


**Вывод**

Для разработки подходит только регион 1, поскольку в остальных двух риски убытков получились слишком высокие: около 4% и 10%.
В регионе 1 с вероятностью 95% прибыль находится между 58.7 млн и 874 млн.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.